# Experiment 3: Basic deep learning
Train 2 deep learning models. 1 on lying videos, 1 on sitting videos: 

### Experiment 3.1 

* input: lying video 
* Output: Left/right leg amplitude/duration (4 scores) 

### Experiment 3.2 

* input: sitting video 
* Output: Left/right arm amplitude/duration (4 scores) 

In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import tensorflow.keras as keras

from src.helpers import read_scores
from src.data_generators import RawDataGenerator
from src.data_selection import MultipleScoreSelector
from src.ai_func import cross_validation_generator
from src.settings import LYING_VIDEOS_DATA_FOLDER, SITTING_VIDEOS_DATA_FOLDER, DATA_FOLDER

In [2]:
%load_ext autoreload
%autoreload 2

### Read metadata

In [60]:
scores_df = read_scores(DATA_FOLDER / 'data_Scoring_DIS_proximal_trunk_V1.1.xlsx')

## Definitions

In [89]:
SCORES_TO_USE = ['D_RLP_R_tD_pscore', 'D_LLP_R_tD_pscore', 'D_RLP_R_tA_pscore', 'D_LLP_R_tA_pscore']
SCORERS = [1, 2, 3]

## Pipeline for training a deep neural network

### Define model architecture (here: simple CNN)

In [62]:
# Generate some dev data to get X shape
dev_selection = selector.transform(scores_df)
dev_generator = RawDataGenerator(dev_selection, videos_folder=LYING_VIDEOS_DATA_FOLDER)
X, y = dev_generator.__getitem__(0)
n_timesteps, n_features = (X.shape[1], X.shape[2])

/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


In [95]:
n_outputs = len(SCORES_TO_USE)

def get_model():
    # simple CNN
    input_layer = keras.layers.Input(shape=(n_timesteps,n_features))
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Flatten()(cnn_layer)
    cnn_layer = keras.layers.Dense(100)(cnn_layer)
    output_layer = keras.layers.Dense(n_outputs)(cnn_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)
get_model().summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 501, 42)]         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 499, 32)           4064      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 497, 32)           3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 248, 32)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 246, 64)           6208      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 244, 64)           12352     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 122, 64)           0   

## Train network

### 

In [108]:
def train_model(train_generator, val_generator):
    model = get_model()
    model.compile(loss='mse', optimizer=keras.optimizers.Adam())
    model.fit(train_generator, epochs=20, validation_data=val_generator)
    return model

def train_cross_val(cross_val):
    y_pred = []
    y_test = []
    for i_split, (train_scores, test_scores) in enumerate(cross_val):
        print(f'Fitting for 5-fold split {i_split}')
        train_scores, val_scores = train_test_split(train_scores, train_size=0.8)
        train_generator = RawDataGenerator(train_scores, videos_folder=LYING_VIDEOS_DATA_FOLDER)
        val_generator = RawDataGenerator(val_scores, videos_folder=LYING_VIDEOS_DATA_FOLDER)
        test_generator = RawDataGenerator(test_scores, videos_folder=LYING_VIDEOS_DATA_FOLDER)
        model = train_model(train_generator, val_generator)
        y_pred.append(model.predict(test_generator))
        y_test.append(test_scores)
    y_pred = np.vstack(y_pred)
    y_test = pd.concat(y_test)
    return y_test, y_pred

def evaluate(y_test, y_pred):
    results = []
    for i_score, column in enumerate(y_test):
        mae = mean_absolute_error(y_test.iloc[:, i_score], y_pred[:, i_score])
        results.append({'score': column, 'mae': mae})
    return pd.DataFrame(results)

In [ ]:
results = []
for scorer in SCORERS:
    print(f'Training model for scorer {scorer}')
    selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=scorer)
    selected_data = selector.transform(scores_df)
    cross_val = cross_validation_generator(selected_data)
    y_test, y_pred = train_cross_val(cross_val)
    results.append((y_test, y_pred))

Training model for scorer 1


/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/20
39/39 [==============================] - 1s 18ms/step - loss: 49823.9648 - val_loss: 1317.1062
Epoch 2/20
39/39 [==============================] - 1s 17ms/step - loss: 775.0305 - val_loss: 202.2708
Epoch 3/20
39/39 [==============================] - 1s 16ms/step - loss: 746.4792 - val_loss: 1186.4116
Epoch 4/20
39/39 [==============================] - 1s 16ms/step - loss: 562.1528 - val_loss: 111.1674
Epoch 5/20
39/39 [==============================] - 1s 16ms/step - loss: 116.3481 - val_loss: 31.1224
Epoch 6/20
39/39 [==============================] - 1s 17ms/step - loss: 40.2595 - val_loss: 19.7177
Epoch 7/20
39/39 [==============================] - 1s 17ms/step - loss: 16.7026 - val_loss: 13.5103
Epoch 8/20
39/39 [==============================] - 1s 17ms/step - loss: 13.7235 - val_loss: 29.7724
Epoch 9/20
39/39 [==============================] - 1s 16ms/step - loss: 12.9333 - val_loss: 15.0687
Epoch 10/20
39/39 [==============================] 

/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows [] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/20
29/29 [==============================] - 1s 20ms/step - loss: 79320.6875 - val_loss: 708.5212
Epoch 2/20
29/29 [==============================] - 0s 16ms/step - loss: 828.4623 - val_loss: 176.5884
Epoch 3/20
29/29 [==============================] - 0s 16ms/step - loss: 147.2329 - val_loss: 173.4036
Epoch 4/20
29/29 [==============================] - 0s 16ms/step - loss: 56.4192 - val_loss: 124.7087
Epoch 5/20
29/29 [==============================] - 0s 16ms/step - loss: 31.1444 - val_loss: 137.3285
Epoch 6/20
29/29 [==============================] - 0s 16ms/step - loss: 23.8281 - val_loss: 115.0125
Epoch 7/20
29/29 [==============================] - 0s 17ms/step - loss: 27.9178 - val_loss: 82.8166
Epoch 8/20
29/29 [==============================] - 0s 16ms/step - loss: 41.7523 - val_loss: 76.7020
Epoch 9/20
29/29 [==============================] - 0s 17ms/step - loss: 37.5724 - val_loss: 60.0239
Epoch 10/20
29/29 [==============================] - 

/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows [] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/20
28/28 [==============================] - 1s 22ms/step - loss: 49551.9062 - val_loss: 3243.8625
Epoch 2/20
28/28 [==============================] - 0s 17ms/step - loss: 1611.1760 - val_loss: 1601.2404
Epoch 3/20
28/28 [==============================] - 0s 17ms/step - loss: 477.5854 - val_loss: 152.3822
Epoch 4/20
28/28 [==============================] - 0s 17ms/step - loss: 113.6408 - val_loss: 84.3758
Epoch 5/20
28/28 [==============================] - 0s 16ms/step - loss: 29.5061 - val_loss: 37.7698
Epoch 6/20
28/28 [==============================] - 0s 17ms/step - loss: 27.2834 - val_loss: 39.5284
Epoch 7/20
28/28 [==============================] - 1s 18ms/step - loss: 19.3339 - val_loss: 36.5278
Epoch 8/20
28/28 [==============================] - 0s 17ms/step - loss: 14.1342 - val_loss: 34.2036
Epoch 9/20
28/28 [==============================] - 1s 20ms/step - loss: 11.7062 - val_loss: 23.6510
Epoch 10/20
28/28 [==============================] -

In [107]:
for scorer, (y_test, y_pred) in zip(SCORERS, results):
    print(f'results for scorer {scorer}')
    print(evaluate(y_test, y_pred))

results for scorer 1
               score       mae
0  D_RLP_R_tD_pscore  7.678542
1  D_LLP_R_tD_pscore  6.814481
2  D_RLP_R_tA_pscore  3.441207
3  D_LLP_R_tA_pscore  5.150673
results for scorer 2
               score       mae
0  D_RLP_R_tD_pscore  4.864221
1  D_LLP_R_tD_pscore  3.773288
2  D_RLP_R_tA_pscore  2.378368
3  D_LLP_R_tA_pscore  2.286805
results for scorer 3
               score       mae
0  D_RLP_R_tD_pscore  3.947798
1  D_LLP_R_tD_pscore  3.033897
2  D_RLP_R_tA_pscore  6.401334
3  D_LLP_R_tA_pscore  5.162821
